# Initialization

In [9]:
%reload_ext autoreload
%autoreload 2

In [10]:
from result_saver import SaverProvider

provider = SaverProvider()

In [11]:
from Scratch import load_calibration_memory

DEVICE = 'ibm_sherbrooke'
OTHER_DATE = '2023-10-30'

all_memories = load_calibration_memory(provider, tobecalib_backend=DEVICE, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


# Functions:

In [12]:
import numpy as np
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

def fit_gmm(IQ_data, scaler):

    data = IQ_data.flatten()
    combined_data = np.column_stack((data.real, data.imag))
    combined_data = scaler.transform(combined_data)

    n_components_range = range(1, 4)  

    lowest_bic = np.infty
    best_gmm = None

    for n_components in n_components_range:

        weights_init = [1]

        # Distribute the remaining weight equally among other components
        if n_components > 1:
            # Initialize weights
            weights_init = np.zeros(n_components)
            primary_weight = 0.95  # Weight for the first component
            weights_init = np.zeros(n_components)
            weights_init[0] = primary_weight
            remaining_weight = (1 - primary_weight) / (n_components-1)
            for i in range(1, n_components):
                weights_init[i] = remaining_weight

        if n_components == 2:
            means_init = np.array([[-1e8, 0], [1e8, 0]])
            # means_init = None
        else:
            means_init = None

        # Fit a Gaussian mixture with EM
        gmm = GaussianMixture(n_components=n_components, init_params='random',
                            covariance_type='tied', random_state=0, weights_init=weights_init, means_init=means_init)
        gmm.fit(combined_data)
        
        # Calculate BIC
        bic = gmm.bic(combined_data)
        
        # Check if this is the lowest BIC so far
        if bic < lowest_bic and np.all([weight < 0.1 or weight > 0.9 for weight in gmm.weights_]):
            lowest_bic = bic
            best_gmm = gmm
    
    return best_gmm

def draw_ellipse(mean, cov, n_std=1, ax=None, **kwargs):
    """Draw an ellipse with a given mean and covariance"""
    ax = ax or plt.gca()

    # Eigenvalues and eigenvectors of the covariance matrix
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * n_std * np.sqrt(vals)
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)

    ax.add_patch(ellipse)


def plot_gmm(gmm, data_2d, n_std=1):
    plt.scatter(data_2d[:, 0], data_2d[:, 1], alpha=0.5)

    # Plot each Gaussian component
    for mean, covar in zip(gmm.means_, gmm.covariances_):
        draw_ellipse(mean, covar, n_std=n_std, alpha=1, edgecolor='red', facecolor='none')

    plt.title("Data and Fitted Gaussian Components")
    plt.xlabel("Real Part")
    plt.ylabel("Imaginary Part")
    plt.show()

# Scale and fit

In [13]:
from soft_info import get_KDEs

kde_dict, scaler_dict = get_KDEs(provider, tobecalib_backend=DEVICE, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


In [14]:
gmm_dict = {}
for qubit in all_memories.keys():
    gmm_dict[qubit] = {}
    scaler = scaler_dict[qubit]
    for mmr in all_memories[qubit].keys():
        data = all_memories[qubit][mmr]
        gmm_dict[qubit][mmr] = fit_gmm(data, scaler)

# takes 3s

# Generate grid

In [54]:
import cpp_soft_info
from Scratch import process_scaler_dict
from scipy.stats import multivariate_normal

num_std_dev = 2
num_points = 300

grid_dict = {}

grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

for qubit_idx, gmms in gmm_dict.items():    
    grid_density_0 = gmms['mmr_0'].score_samples(grid_points).reshape(grid_x.shape)
    grid_density_1 = gmms['mmr_1'].score_samples(grid_points).reshape(grid_x.shape)

    grid_dict[qubit_idx] = cpp_soft_info.GridData(grid_x, grid_y, grid_density_0, grid_density_1)


# Scaler dict 
processed_scaler_dict = process_scaler_dict(scaler_dict)

# takes 3s

In [55]:
import cpp_soft_info
from Scratch import process_scaler_dict
from scipy.stats import multivariate_normal

grid_dict_no_outliers = {}

grid_range_real = np.linspace(-num_std_dev, num_std_dev, num_points)
grid_range_imag = np.linspace(-num_std_dev, num_std_dev, num_points)
grid_x, grid_y = np.meshgrid(grid_range_real, grid_range_imag)
grid_points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T

for qubit_idx, gmms in gmm_dict.items():    
    for i, gmm in enumerate(gmms.values()):
        max_weight_index = np.argmax(gmm.weights_)
        mvn = multivariate_normal(mean=gmm.means_[max_weight_index], cov=gmm.covariances_)
        grid_density = mvn.pdf(grid_points).reshape(grid_x.shape)
        if i == 0:
            grid_density_0 = grid_density
        else:
            grid_density_1 = grid_density

    grid_dict_no_outliers[qubit_idx] = cpp_soft_info.GridData(grid_x, grid_y, grid_density_0, grid_density_1)


# Scaler dict 
processed_scaler_dict = process_scaler_dict(scaler_dict)

# takes 0.6s

# Decoding

In [56]:
from soft_info import RepCodeIQSimulator

DISTANCE = 7
ROUNDS = 7
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

In [57]:
SHOTS = int(1e4)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.3

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

100%|██████████| 8871/8871 [00:03<00:00, 2825.42it/s]


In [58]:
import pymatching
import stim

model = simulator.stim_circ.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

### SOFT INFO DECODING

In [59]:
import cpp_soft_info

_DETAILED = False

matching = pymatching.Matching.from_detector_error_model(model)
result = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                           common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                            p_offset = 1, p_multiplicator = 1, _ntnn_edges = not _RESETS)

matching = pymatching.Matching.from_detector_error_model(model)
result_extreme = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data_extreme,
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                             simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                             common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                              p_offset = 1, p_multiplicator = 1, _ntnn_edges = not _RESETS)

print("num_errors:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 132 out of 10000 shots for _RESETS = False
num_errors EXTREME: 272 out of 10000 shots for _RESETS = False


In [63]:
import cpp_soft_info

_DETAILED = False

matching = pymatching.Matching.from_detector_error_model(model)
result = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, grid_dict,
                                           simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                           common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                            p_offset = 1, p_multiplicator = 1, _ntnn_edges = not _RESETS)

matching = pymatching.Matching.from_detector_error_model(model)
result_extreme = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data_extreme,
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, grid_dict_no_outliers,
                                             simulator.processed_scaler_dict, p_data=-1, p_mixed=-1,
                                             common_measure=-1, _adv_probs=not _RESETS, _bimodal=_RESETS, merge_strategy = "replace", _detailed=_DETAILED,
                                              p_offset = 1, p_multiplicator = 1, _ntnn_edges = not _RESETS)

print("num_errors:", result.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 144 out of 10000 shots for _RESETS = False
num_errors EXTREME: 747 out of 10000 shots for _RESETS = False


### INFORMED DECODING

In [64]:
p_meas = -1
# p_meas = 30e-2 

matching = pymatching.Matching.from_detector_error_model(model)
result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = True)

matching = pymatching.Matching.from_detector_error_model(model)
result_informed_extreme = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data_extreme[:],
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                             p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = True)

print("num_errors:", result_informed.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_informed_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)
         
# takes 1s

num_errors: 166 out of 10000 shots for _RESETS = False
num_errors EXTREME: 2435 out of 10000 shots for _RESETS = False


In [69]:
p_meas = -1
# p_meas = 30e-2 

matching = pymatching.Matching.from_detector_error_model(model)
result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, grid_dict_no_outliers, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = True)

matching = pymatching.Matching.from_detector_error_model(model)
result_informed_extreme = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data_extreme[:],
                                             ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, grid_dict_no_outliers, simulator.processed_scaler_dict,
                                             p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = True)

print("num_errors:", result_informed.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_informed_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)
         
# takes 1s

num_errors: 162 out of 10000 shots for _RESETS = False
num_errors EXTREME: 857 out of 10000 shots for _RESETS = False


### FLAT DECODING

In [66]:
matching = pymatching.Matching.from_detector_error_model(model)
result_flat = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED)

matching = pymatching.Matching.from_detector_error_model(model)
result_flat_extreme = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data_extreme,
                                                         ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                                            simulator.processed_scaler_dict, _detailed=_DETAILED)

print("num_errors:", result_flat.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_flat_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 196 out of 10000 shots for _RESETS = False
num_errors EXTREME: 3740 out of 10000 shots for _RESETS = False


In [67]:
matching = pymatching.Matching.from_detector_error_model(model)
result_flat = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, grid_dict_no_outliers,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED)

matching = pymatching.Matching.from_detector_error_model(model)
result_flat_extreme = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data_extreme,
                                                         ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, grid_dict_no_outliers,
                                                            simulator.processed_scaler_dict, _detailed=_DETAILED)

print("num_errors:", result_flat.num_errors, "out of", len(IQ_data), "shots for _RESETS =", _RESETS)
print("num_errors EXTREME:", result_flat_extreme.num_errors, "out of", len(IQ_data_extreme), "shots for _RESETS =", _RESETS)

num_errors: 198 out of 10000 shots for _RESETS = False
num_errors EXTREME: 1549 out of 10000 shots for _RESETS = False
